In [43]:
import random
import time
import os
import glob 
import networkx as nx
import numpy as np
from copy import deepcopy
from tabulate import tabulate
from matplotlib import pyplot as plt

RESULTS_PATH = "resultsGA"

if not os.path.exists(RESULTS_PATH):
    os.makedirs(RESULTS_PATH)

# Omogućava prikaz grafikona unutar Notebook-a
%matplotlib inline

In [ ]:
def maxNumOfColours(graph):
    maxDegree = 0
    for i in graph:
        degree = len(list(graph.neighbors(i)))
        if degree > maxDegree:
            maxDegree = degree
    return maxDegree



def readGraph(filePath):
    graph = nx.Graph()
    edges = []
    
    with open(filePath, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts: continue
            
            # Preskoči meta-podatke (p edge 11 55)
            if parts[0] in ['p', 'c', 'edge']:
                continue
            
            # Čitaj ivice
            try:
                if len(parts) >= 2:
                    # Uzimamo prva dva broja u redu, šta god da su
                    u, v = int(parts[-2]), int(parts[-1])
                    edges.append((u, v))
            except ValueError:
                continue

    # Ključni momenat: NetworkX automatski kreira čvorove na osnovu ivica
    graph.add_edges_from(edges)
    
    # Ispisujemo tačan broj učitanih ivica radi tvoje provere
    # Ako ovde ne piše 55 za K11, fajl ti ne valja!
    return graph


def getFitness(chromosome, edges, maxNode):
    # Ako dužine nisu iste, nešto ozbiljno ne valja sa učitavanjem
    if len(chromosome) != len(edges):
        return 999999

    conflicts = 0
    # nodeMap: čvor -> lista boja ivica koje su povezane s njim
    nodeMap = {}

    for i in range(len(edges)):
        u, v = edges[i]
        color = chromosome[i]
        
        # Registruj boju za oba čvora ivice
        for node in [u, v]:
            if node not in nodeMap:
                nodeMap[node] = []
            nodeMap[node].append(color)

    # Sada brojimo konflikte
    for node, colors in nodeMap.items():
        # Brojimo koliko se koja boja puta pojavljuje u ovom čvoru
        from collections import Counter
        counts = Counter(colors)
        
        for color, count in counts.items():
            if count > 1:
                # Ako se ista boja javi 'count' puta, to je:
                # npr. 2 ivice = 1 konflikt, 3 ivice = 3 konflikta, 4 ivice = 6...
                conflicts += (count * (count - 1)) // 2
                
    return conflicts

In [ ]:
def selectionTournament(population, fitnesses, k=3):
    participants = random.sample(list(zip(population, fitnesses)), k)
    return min(participants, key=lambda x: x[1])[0]

def selectionRoulette(population, fitnesses):
    # Što manji fitnes, to veća šansa (proporcionalno 1/f)
    invertedFitness = [1.0 / (f + 1e-6) for f in fitnesses]
    total = sum(invertedFitness)
    pick = random.uniform(0, total)
    current = 0
    for i, f in enumerate(invertedFitness):
        current += f
        if current > pick:
            return population[i]
    return population[0]

def selectionRank(population, fitnesses):
    popSize = len(population)
    sortedIndices = np.argsort(fitnesses)
    ranks = np.empty(popSize)
    for rank, idx in enumerate(range(popSize)):
        ranks[sortedIndices[idx]] = popSize - rank
    totalRank = sum(ranks)
    pick = random.uniform(0, totalRank)
    current = 0
    for i in range(popSize):
        current += ranks[i]
        if current > pick: return population[i]
    return population[0]



In [ ]:
def crossoverUniform(p1, p2):
    return [p1[i] if random.random() < 0.5 else p2[i] for i in range(len(p1))]

def crossoverSinglePoint(p1, p2):
    point = random.randint(1, len(p1) - 1)
    return p1[:point] + p2[point:]

def crossoverTwoPoint(p1, p2):
    size = len(p1)
    if size < 3: return crossoverSinglePoint(p1, p2)
    pt1 = random.randint(1, size - 2)
    pt2 = random.randint(pt1 + 1, size - 1)
    return p1[:pt1] + p2[pt1:pt2] + p1[pt2:]



In [47]:
def mutationSmart(chromosome, edges, maxNode, numColors):
    idx = random.randrange(len(chromosome))
    u, v = edges[idx]
    forbidden = set()
    
    # Prolazimo kroz sve ivice da nađemo susedne (optimizaćemo ovo kasnije)
    for i, (n1, n2) in enumerate(edges):
        if i != idx and (n1 in (u, v) or n2 in (u, v)):
            forbidden.add(chromosome[i])
            
    # Koristimo isključivo DOZVOLJENI opseg boja (numColors)
    available = list(set(range(1, numColors + 1)) - forbidden)
    
    if available:
        chromosome[idx] = random.choice(available)
    else:
        # Ako nema slobodne boje u dozvoljenom opsegu, 
        # uzmi bilo koju iz DOZVOLJENOG opsega, ne dodaj nove!
        chromosome[idx] = random.randint(1, numColors) 
    return chromosome


def mutationRandom(chromosome, edges, maxNode,numColors):
    idx = random.randrange(len(chromosome))
    chromosome[idx] = random.randint(1, numColors)
    return chromosome

#def mutationSwap(chromosome, edges, maxNode, delta):
#    idx1, idx2 = random.sample(range(len(chromosome)), 2)
#    chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]
#    return chromosome
#KAO NIJE KORISNO ZA MINIMUM EDGE COLORING


def mutationConflict(chromosome, edges, maxNode, numColors):
    # 1. Mapiramo koji čvor ima koje boje na svojim ivicama
    # Ovo nam pomaže da brzo vidimo gde su duplikati boja
    node_colors = {i: [] for i in range(1, maxNode + 1)}
    for i, (u, v) in enumerate(edges):
        node_colors[u].append(chromosome[i])
        node_colors[v].append(chromosome[i])
        
    # 2. Pronalazimo indekse svih ivica koje učestvuju u konfliktu
    conflicted_indices = []
    for i, (u, v) in enumerate(edges):
        # Ako se trenutna boja ivice pojavljuje više od jednom kod čvora u ili v
        if node_colors[u].count(chromosome[i]) > 1 or node_colors[v].count(chromosome[i]) > 1:
            conflicted_indices.append(i)
            
    # 3. Akcija:
    if not conflicted_indices:
        # Ako nema konflikata (čestitamo!), uradi malu nasumičnu promenu
        idx = random.randrange(len(chromosome))
        chromosome[idx] = random.randint(1, numColors)
    else:
        # Izaberi jednu od ivica koje STVARNO prave problem i promeni joj boju
        idx = random.choice(conflicted_indices)
        chromosome[idx] = random.randint(1, numColors)
    
    return chromosome



In [48]:
def runGaModular(graph, maxIters, popSize, selFunc, crossFunc, mutFunc, numColours):
    edges = list(graph.edges())
    if not edges: return [], 0, []
    
    # maxNode nam treba za mutationConflict da zna veličinu rečnika čvorova
    maxNode = max(graph.nodes())
    numEdges = len(edges)
    
    # Inicijalizacija: Sada strogo poštujemo numColours
    population = [[random.randint(1, numColours) for _ in range(numEdges)] for _ in range(popSize)]
    
    history = []
    
    for gen in range(maxIters):
        fitnesses = [getFitness(c, edges, maxNode) for c in population]
        bestFit = min(fitnesses)
        history.append(bestFit)
        
        if bestFit == 0:
            # Popunjavamo istoriju do kraja radi grafikona
            history.extend([0] * (maxIters - len(history)))
            break
            
        # Elitizam: Sortiramo i uzimamo 2 najbolja
        popWithFit = sorted(zip(population, fitnesses), key=lambda x: x[1])
        newPop = [popWithFit[0][0], popWithFit[1][0]]
        
        while len(newPop) < popSize:
            # 1. Poboljšanje: Izbegavamo da p1 i p2 budu ista jedinka (ako je moguće)
            p1_idx = population.index(selFunc(population, fitnesses))
            p2_idx = population.index(selFunc(population, fitnesses))
            
            # Ako je populacija raznolika, probaj da nađeš različite roditelje
            attempts = 0
            while p1_idx == p2_idx and attempts < 5:
                p2_idx = population.index(selFunc(population, fitnesses))
                attempts += 1
            
            p1, p2 = population[p1_idx], population[p2_idx]
            
            # 2. Ukrštanje
            child = crossFunc(p1, p2)
            
            # 3. Mutacija (povećana šansa jer koristimo pametne mutacije)
            # Za Edge Coloring, 0.3 - 0.4 je odličan ratio
            if random.random() < 0.35:
                # mutationConflict i mutationSmart sada dobijaju sve što im treba
                child = mutFunc(child, edges, maxNode, numColours)
                
            newPop.append(child)
            
        population = newPop
        
    # Finalna provera najboljeg
    finalFits = [getFitness(c, edges, maxNode) for c in population]
    bestIdx = np.argmin(finalFits)
    
    return population[bestIdx], finalFits[bestIdx], history

In [49]:
def processAndVisualize(filePath):
    graph = readGraph(filePath)

    # >>> OVDE DODAJ OVE LINIJE <<<
    print(f"\n" + "="*40)
    print(f"DEBUG ZA FAJL: {os.path.basename(filePath)}")
    print(f"Stvarni broj čvorova: {graph.number_of_nodes()}")
    print(f"Stvarni broj ivica: {graph.number_of_edges()}")
    # >>> KRAJ DODATKA <<<

    fileName = os.path.basename(filePath)
    delta = maxNumOfColours(graph)

    # Dodaj i ovo da proverimo šta je funkcija izračunala za Deltu
    print(f"Izračunata Delta: {delta}")
    print("="*40 + "\n")

    if graph.number_of_edges() == 0: return

    
    f_lower = fileName.lower()
    type_info = "Unknown Graph Type"
    theory_note = ""
    is_bipartite = nx.is_bipartite(graph)
    if "completegraph" in f_lower or "k_" in f_lower:
        try:
            n = int(''.join(filter(str.isdigit, fileName)))
            type_info = f"Complete Graph K({n})"
            if n % 2 == 0: theory_note = f"Class 1 (Expected χ' = Δ = {n-1})"
            else: theory_note = f"Class 2 (Expected χ' = Δ + 1 = {n})"
        except: type_info = "Complete Graph"
    elif "bipartite" in f_lower or is_bipartite:
        type_info = "Bipartite Graph"
        theory_note = f"Always Class 1 (Expected χ' = Δ = {delta})"
    elif "petersen" in f_lower or "snark" in f_lower or "flower" in f_lower:
        type_info = "SNARK / Petersen Graph"
        theory_note = f"Class 2 (Expected χ' = Δ + 1 = {delta+1})"
    elif "regular" in f_lower:
        type_info = f"Random Regular Graph (d={delta})"
        theory_note = "Most are Class 1"
    elif "graph" in f_lower:
        type_info = "Random G(n,p) Graph"
        theory_note = "Vizing's Theorem: Class 1 or 2"

    print(f"\n>>> ANALYZING: {fileName}")
    print(f"[TYPE]: {type_info}")
    if theory_note: print(f"[THEORY]: {theory_note}")

    selections = [selectionTournament, selectionRoulette, selectionRank]
    crossovers = [crossoverSinglePoint, crossoverUniform, crossoverTwoPoint]
    mutations = [mutationRandom, mutationSmart, mutationConflict]

    
    def run_tests(num_cols):
        res_list = []
        for sel in selections:
            for cross in crossovers:
                for mut in mutations:
                    sName = sel.__name__.replace('selection', '')
                    cName = cross.__name__.replace('crossover', '')
                    mName = mut.__name__.replace('mutation', '')
                    t0 = time.perf_counter()
                    sol, fit, hist = runGaModular(graph, 500, 100, sel, cross, mut, num_cols)
                    dt = time.perf_counter() - t0
                    res_list.append({'selection': sName, 'crossover': cName, 'mutation': mName,
                                     'fit': fit, 'time': dt, 'hist': hist, 'sol': sol, 'cols': num_cols})
        return res_list

    # Prvi krug
    allResults = run_tests(delta)
    allResults.sort(key=lambda x: (x['fit'], x['time']))
    
    found_zero = allResults[0]['fit'] == 0
    secondAttemptResults = []

    if not found_zero:
        print(f"!!! Nema nule sa {delta} boja. Pokrećem Delta + 1 = {delta+1}...")
        secondAttemptResults = run_tests(delta + 1)
        secondAttemptResults.sort(key=lambda x: (x['fit'], x['time']))

    
    def prepare_table(results):
        data = []
        for i, r in enumerate(results):
            data.append([i, r['selection'], r['crossover'], r['mutation'], r['fit'], r['cols'], f"{r['time']:.6f}"])
        return data

    headers = ["Rank", "selection", "crossover", "mutation", "num of conf", "colours", "time"]
    
    # Konzola: Top 3
    print(f"Top 3 Combinations for {fileName}:")
    print(tabulate(prepare_table(allResults if found_zero else secondAttemptResults)[:3], headers=headers, tablefmt="fancy_grid"))

    # Fajl: format sa dopisivanjem rezultata
    txtPath = os.path.join(RESULTS_PATH, fileName) 
    with open(txtPath, 'w', encoding='utf-8') as f:
        f.write(f"Graph: {fileName} | Type: {type_info}\nTheory: {theory_note}\nMax Degree (Delta): {delta}\n\n")
        f.write("--- RESULTS FOR DELTA ---\n")
        f.write(tabulate(prepare_table(allResults), headers=headers, tablefmt="fancy_grid"))
        if secondAttemptResults:
            f.write("\n\n--- RESULTS FOR DELTA + 1 ---\n")
            f.write(tabulate(prepare_table(secondAttemptResults), headers=headers, tablefmt="fancy_grid"))
        
        if found_zero: f.write(f"\n\nRESULT: Class 1 found (Index = {delta})")
        elif secondAttemptResults and secondAttemptResults[0]['fit'] == 0: f.write(f"\n\nRESULT: Class 2 found (Index = {delta+1})")
        else: f.write(f"\n\nRESULT: GA failed to find 0 conflicts.")

    
    best = allResults[0] if found_zero or not secondAttemptResults else secondAttemptResults[0]
    plt.figure(figsize=(16, 7))
    plt.subplot(1, 2, 1)
    pos = nx.spring_layout(graph, seed=42)
    #nx.draw(graph, pos, edge_color=best['sol'], width=2.5, with_labels=True, node_color='lightblue', edge_cmap=plt.cm.rainbow)
    edges_list = list(graph.edges())
    nx.draw(graph, pos, 
        edgelist=edges_list,        # Eksplicitno kažemo koje ivice crtamo
        edge_color=best['sol'],     # Boje iz pobedničkog hromozoma
        width=3.0,                  # Malo deblje ivice da se bolje vidi boja
        with_labels=True, 
        node_color='lightblue', 
        node_size=500,
        edge_cmap=plt.cm.rainbow)   # Mapira brojeve 1, 2, 3... u dugu
        #edge_vmin=1,                # Najmanja moguća boja
        #edge_vmax=num_cols)         # Najveća moguća boja (numColours)
    plt.title(f"Winner (Rank 0):\n{best['selection']} + {best['crossover']} + {best['mutation']}")

    plt.subplot(1, 2, 2)
    plot_data = allResults[:3] if found_zero else secondAttemptResults[:3]
    for i, res in enumerate(plot_data):
        label_name = f"Rank {i}: {res['selection']} + {res['crossover']} + {res['mutation']}"
        plt.plot(res['hist'], label=label_name, linewidth=2.5)
    
    plt.xlabel("Generations (Algorithm Progress)", fontsize=11)
    plt.ylabel("Number of Conflicts (Fitness)", fontsize=11)
    plt.title(f"Convergence Race: Top 3 Algorithms for {fileName}", fontsize=13)
    plt.legend(loc='upper right', fontsize='small')
    plt.grid(True, linestyle=':', alpha=0.8)
    plt.tight_layout()
    plt.show()
    plt.close('all')
    print(f"Full table saved to: {txtPath}")

In [50]:
#Novo ucitavanje fajla

targetFolders = ['experimentalTests']

for folder in targetFolders:
    #kreiramo putanju za pretragu
    searchPath = os.path.join('tests',folder,"*.txt")
    allFiles = sorted(glob.glob(searchPath))

    if not allFiles:
        print(f"No files found in: tests/{folder}")
        continue

    print(f"\n{'#'*40}\n# ENTERING FOLDER: {folder}\n{'#'*40}")

    for f in allFiles:
        #fileName = os.path.basename(f)
        processAndVisualize(f)


########################################
# ENTERING FOLDER: experimentalTests
########################################

DEBUG ZA FAJL: completeBipartite10.txt
Stvarni broj čvorova: 20
Stvarni broj ivica: 100
Izračunata Delta: 10


>>> ANALYZING: completeBipartite10.txt
[TYPE]: Bipartite Graph
[THEORY]: Always Class 1 (Expected χ' = Δ = 10)


KeyboardInterrupt: 